In [2]:
import psycopg2
from psycopg2 import sql
import pandas as pd

In [4]:
# Credenciales de la base de datos
dbname = "DBInmerso"
user = "postgres"
password = "postgres"
host = "localhost"  # Cambia esto según tu configuración
port = "5432"

# Crear la cadena de conexión
connection_string = f"dbname={dbname} user={user} password={password} host={host} port={port}"


try:
    with psycopg2.connect(connection_string) as connection:
        # Consulta para concatenar las tablas
    
        query_concatenar_3 = """SELECT 
                                s.id_solicitud,
                                sc.index_,
                                sc.score, 
                                s.primera_opcion,
                                s.fecha_1op,
                                s.segunda_opcion, 
                                s.fecha_2op,
                                s.tercera_opcion,
                                s.fecha_3op
                                FROM solicitudes s
                                JOIN scoring sc ON s.id_solicitud = sc.id_solicitud;
                                """
        query_concatenar_4 = """SELECT 
                                d.id_hotel,
                                d.fecha_disponibilidad_hab,
                                d.num_hab_disp
                                FROM disponibilidad d;
                                """


        # Crear un DataFrame con los resultados de la consulta concatenada
    
        df_3 = pd.read_sql_query(query_concatenar_3, connection)
        df_4 = pd.read_sql_query(query_concatenar_4, connection)

except psycopg2.Error as e:
    print(f"Error al conectar a la base de datos: {e}")


In [5]:
for index, row in df_3.iterrows():
    hotel_match = (df_4['id_hotel'] == row['primera_opcion']) & (df_4['fecha_disponibilidad_hab'] == row['fecha_1op']) & (df_4['num_hab_disp'] > 0)
    
    if hotel_match.all():
        df_3.loc[index, 'asignacion_hotel'] = row['primera_opcion']
        df_3.loc[index, 'asignacion_fecha'] = row['fecha_1op']
        df_4.loc[hotel_match, 'num_hab_disp'] -= 1
        df_3.loc[index, 'id_resolucion'] = 3
    else:
        hotel_match = (df_4['id_hotel'] == row['segunda_opcion']) & (df_4['fecha_disponibilidad_hab'] == row['fecha_2op']) & (df_4['num_hab_disp'] > 0)
        if hotel_match.all():
            df_3.loc[index, 'asignacion_hotel'] = row['segunda_opcion']
            df_3.loc[index, 'asignacion_fecha'] = row['fecha_2op']
            df_4.loc[hotel_match, 'num_hab_disp'] -= 1
            df_3.loc[index, 'id_resolucion'] = 3
        else:
            hotel_match = (df_4['id_hotel'] == row['tercera_opcion']) & (df_4['fecha_disponibilidad_hab'] == row['fecha_3op']) & (df_4['num_hab_disp'] > 0)
            if hotel_match.all():
                df_3.loc[index, 'asignacion_hotel'] = row['tercera_opcion']
                df_3.loc[index, 'asignacion_fecha'] = row['fecha_3op']
                df_4.loc[hotel_match, 'num_hab_disp'] -= 1
                df_3.loc[index, 'id_resolucion'] = 3
            else:
                df_3.loc[index, 'id_resolucion'] = 2

In [6]:
df_3

,id_solicitud,index_,score,primera_opcion,fecha_1op,segunda_opcion,fecha_2op,tercera_opcion,fecha_3op,id_resolucion
0,39426,1,76.500000,Melilla,2024-12-10,Guadalajara,2024-07-27,Ceuta,2024-03-01,2.0
1,273,2,74.500000,Jaén,2024-09-05,Murcia,2024-05-26,Sevilla,2024-08-24,2.0
2,372,3,73.507326,Castellón,2024-04-03,Navarra,2024-02-28,Girona,2024-03-14,2.0
3,51535,4,72.578755,Cáceres,2024-01-07,Ceuta,2024-03-20,Las Palmas,2024-03-14,2.0
4,47129,5,71.435897,Lleida,2024-01-05,Córdoba,2024-07-03,Pontevedra,2024-01-25,2.0
...,...,...,...,...,...,...,...,...,...,...
1199,721,1200,10.642857,Córdoba,2024-04-02,Álava,2024-04-05,Palencia,2024-12-29,2.0
1200,368,1201,8.928571,La Rioja,2024-11-11,Cáceres,2024-06-15,Toledo,2024-12-16,2.0
1201,221,1202,6.500000,Guipúzcoa,2024-07-19,Toledo,2024-05-08,La Coruña,2024-03-19,2.0
1202,191,1203,4.935897,Cáceres,2024-03-27,Murcia,2024-03-25,Zaragoza,2024-04-24,2.0
